<a href="https://colab.research.google.com/github/ma2sevich222/raw_stock_data_researches/blob/main/adaptive_median%20as%20position%20hold%20indicator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
import pandas as pd
import io
import plotly.express as px
import numpy as np
from numpy import linalg as la
import plotly.graph_objects as go

In [ ]:
uploaded = files.upload()

In [ ]:
df=pd.read_csv(io.BytesIO(uploaded['GC_30min.csv']) )

In [ ]:
df = df[32014:]

In [ ]:
df

,Datetime,Open,High,Low,Close,Volume
32014,2021-11-01 01:00:00,1782.3,1783.7,1782.0,1782.7,580.0
32015,2021-11-01 01:30:00,1782.5,1784.1,1781.2,1783.5,1618.0
32016,2021-11-01 02:00:00,1783.5,1785.7,1782.8,1785.3,2123.0
32017,2021-11-01 02:30:00,1785.4,1787.6,1785.4,1786.9,1833.0
32018,2021-11-01 03:00:00,1786.9,1788.5,1786.7,1787.1,1963.0
...,...,...,...,...,...,...
41230,2022-08-24 12:00:00,1757.2,1758.2,1754.8,1757.9,3813.0
41231,2022-08-24 12:30:00,1757.9,1759.9,1757.1,1758.8,4533.0
41232,2022-08-24 13:00:00,1758.6,1760.2,1755.7,1758.6,5265.0
41233,2022-08-24 13:30:00,1758.6,1759.2,1756.2,1756.3,3452.0


In [ ]:
close = df["Close"].values

In [ ]:
nSamples = close.shape[0]
nDim = 1
w = 0
curent_signal = 0
mdks = 0

In [ ]:
Signals = []
mean_close = []

curent_signal = 0
treshhold = 0.0002
upper_treshold = 0.01


In [ ]:
for iter in range(len(close)):
   
    cnt = iter+1
    x = close[iter]
    #x = x.reshape(nDim,1) 
    w = w + (1000/(1 + iter)) * np.sign( x- w) 
    mdks = w
    if x - w > treshhold * w :
      if x - w > upper_treshold * w:
        curent_signal = 0
        w=x
      else:
        curent_signal = 1

    elif x - w < -treshhold * w :
      if x - w < -upper_treshold * w:
          curent_signal = 0
          w=x
      else:
        curent_signal = -1
    else:
      curent_signal = 0
    
    
    Signals.append(curent_signal)
    mean_close.append(mdks)


In [ ]:
df["Signal"] = Signals

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index, y=df['Close'].values,
                    mode='lines',
                    name='close'))


fig.add_trace(go.Scatter(x=df.index, y=mean_close,
                    mode='lines',
                    name='mean_close'))

fig.add_trace(go.Bar(
    x=df.index[df['Signal'] == -1].tolist(),
    y=[df['Close'].max() for i in range(len(df.index[df['Signal'] == -1].tolist()))],
    name='down',
    marker_color='red', opacity=1, 
))
fig.add_trace(go.Bar(
    x=df.index[df['Signal'] == 1].tolist(),
    y=[df['Close'].max() for i in range(len(df.index[df['Signal'] == 1].tolist()))],
    name='up',
    marker_color='green',opacity=0.5, 
))



fig.add_trace(go.Bar(
    x=df.index[df['Signal'] == 0].tolist(),
    y=[df['Close'].max() for i in range(len(df.index[df['Signal'] == 0].tolist()))],
    name='cash',
    marker_color='blue',opacity=0.5, 
))

'''fig.update_xaxes(title_text='Index')
fig.update_yaxes(title_text='Close')
for i in range(0,len(df2),440):
  fig.add_vrect(x0=i, x1=i+1, annotation_text="forward end")'''


#fig.add_vrect(x0=1760, x1=1761, annotation_text="forward end")'''
fig.update_layout(yaxis=dict(range=[df['Close'].min()-1,df['Close'].max()+1]))
fig.update_layout(title_text='Adaptive_median')
fig.show()

In [ ]:
df=df.reset_index(drop=True)

In [ ]:
df=df[600:]

In [ ]:
final_df=df.reset_index(drop=True)

In [ ]:
final_df

,Datetime,Open,High,Low,Close,Volume,Signal
0,2021-12-08 14:00:00,1784.4,1786.5,1782.1,1784.0,4515.0,0
1,2021-12-08 14:30:00,1784.1,1784.2,1780.8,1782.8,4232.0,0
2,2021-12-08 15:00:00,1782.7,1783.9,1780.1,1782.0,6638.0,1
3,2021-12-08 15:30:00,1782.0,1785.6,1781.2,1782.6,6081.0,0
4,2021-12-08 16:00:00,1782.5,1783.3,1780.6,1783.0,3846.0,-1
...,...,...,...,...,...,...,...
8016,2022-08-24 12:00:00,1757.2,1758.2,1754.8,1757.9,3813.0,1
8017,2022-08-24 12:30:00,1757.9,1759.9,1757.1,1758.8,4533.0,1
8018,2022-08-24 13:00:00,1758.6,1760.2,1755.7,1758.6,5265.0,1
8019,2022-08-24 13:30:00,1758.6,1759.2,1756.2,1756.3,3452.0,1


In [ ]:
final_df.to_csv("new_algo_Adaptive_mean_GC_30.csv")